In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df['text'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emai

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df=df.dropna()

In [7]:
X = df.drop('label',axis=1)
y= df['label']
X.shape

(18285, 4)

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [9]:
voc_size=5000

In [10]:
messages=X.copy()

In [11]:
messages.reset_index(inplace=True)

In [12]:
import nltk
import re
from nltk.corpus import stopwords

In [13]:
#nltk.download('stopwords')

In [14]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [15]:
X.shape

(18285, 4)

In [16]:
X = X[~X['title'].str.isnumeric()]


In [17]:
X.shape

(18285, 4)

In [18]:
print(messages.columns)


Index(['index', 'id', 'title', 'author', 'text'], dtype='object')


In [19]:
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[1:5]

[[500, 1454, 2431, 917, 616, 4403, 1553],
 [841, 4372, 3833, 2287],
 [4208, 4444, 3174, 4266, 3342, 3956],
 [896, 616, 59, 4912, 1359, 4566, 616, 4448, 4296, 347]]

One-hot encoding is a simple way to represent words numerically, but it doesn't capture semantic similarities between words.

<h3>Using Word Embeddings and LSTM
<h5>Add Word Embeddings just add zero in the beginning or the End


In [21]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs[1:10])

[[ 500 1454 2431  917  616 4403 1553    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [ 841 4372 3833 2287    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [4208 4444 3174 4266 3342 3956    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [ 896  616   59 4912 1359 4566  616 4448 4296  347    0    0    0    0
     0    0    0    0    0    0]
 [4927 2048 2412 3988 4755 1012 4435 3412 1829 1696 3970  820 2641 2562
  1553    0    0    0    0    0]
 [ 410 2190 3918 3416  791  332 2190 3583 3304  750  875    0    0    0
     0    0    0    0    0    0]
 [3318 1855 4370  719 2494 3514 1012 1425 3304  750  875    0    0    0
     0    0    0    0    0    0]
 [4843 4813 2497   66 3116 4129 2821 2605 1012 1043    0    0    0    0
     0    0    0    0    0    0]
 [1370  120 3180 3019  306 3793  451 4785    0    0    0    0    0    0
     0    0    0    0    0    0]]


In [22]:
embedded_vector_features=40
model=Sequential()

Applying LSTM on the Data 

In [23]:
model.add(Embedding(voc_size,embedded_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# new_learning_rate = 0.001
# model.compile(optimizer=SGD(learning_rate=new_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [24]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [25]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [27]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10


192/192 [==============================] - 26s 83ms/step - loss: 0.3171 - accuracy: 0.8444 - val_loss: 0.2030 - val_accuracy: 0.9137
Epoch 2/10
192/192 [==============================] - 15s 77ms/step - loss: 0.1350 - accuracy: 0.9482 - val_loss: 0.2002 - val_accuracy: 0.9170
Epoch 3/10
192/192 [==============================] - 15s 76ms/step - loss: 0.0881 - accuracy: 0.9682 - val_loss: 0.2241 - val_accuracy: 0.9160
Epoch 4/10
192/192 [==============================] - 13s 67ms/step - loss: 0.0629 - accuracy: 0.9777 - val_loss: 0.2875 - val_accuracy: 0.9118
Epoch 5/10
192/192 [==============================] - 12s 65ms/step - loss: 0.0447 - accuracy: 0.9851 - val_loss: 0.3171 - val_accuracy: 0.9112
Epoch 6/10
192/192 [==============================] - 13s 66ms/step - loss: 0.0284 - accuracy: 0.9906 - val_loss: 0.3799 - val_accuracy: 0.9074
Epoch 7/10
192/192 [==============================] - 12s 63ms/step - loss: 0.0237 - accuracy: 0.9931 - val_loss: 0.3294 - val_accuracy: 0.9056
Epo

<h3>Adding Dropout Layer to the Model

In [28]:
from tensorflow.keras.layers import Dropout
import tensorflow as tf
## Creating model
embedding_vector_features=40
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model2.add(Dropout(0.3))
model2.add(LSTM(100))
model2.add(Dropout(0.3))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [29]:
early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience= 20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [30]:
### Finally Training
model_history=model2.fit(X_train,y_train,validation_split=0.33,batch_size=32,epochs=30,callbacks=early_stopping)

Epoch 1/30


257/257 [==============================] - 15s 37ms/step - loss: 0.3191 - accuracy: 0.8386 - val_loss: 0.2042 - val_accuracy: 0.9147
Epoch 2/30
257/257 [==============================] - 8s 31ms/step - loss: 0.1417 - accuracy: 0.9453 - val_loss: 0.1978 - val_accuracy: 0.9166
Epoch 3/30
257/257 [==============================] - 8s 31ms/step - loss: 0.0966 - accuracy: 0.9671 - val_loss: 0.2760 - val_accuracy: 0.9063
Epoch 4/30
257/257 [==============================] - 8s 31ms/step - loss: 0.0689 - accuracy: 0.9776 - val_loss: 0.3092 - val_accuracy: 0.9080
Epoch 5/30
257/257 [==============================] - 8s 32ms/step - loss: 0.0584 - accuracy: 0.9807 - val_loss: 0.2894 - val_accuracy: 0.9092
Epoch 6/30
257/257 [==============================] - 8s 32ms/step - loss: 0.0446 - accuracy: 0.9851 - val_loss: 0.4332 - val_accuracy: 0.9080
Epoch 7/30
257/257 [==============================] - 9s 34ms/step - loss: 0.0353 - accuracy: 0.9883 - val_loss: 0.3434 - val_accuracy: 0.9068
Epoch 8/3

In [31]:
y_pred=model2.predict(X_test)

189/189 [==============================] - 2s 9ms/step


In [32]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve
from sklearn.metrics import confusion_matrix

In [33]:
confusion_matrix(y_test,y_pred)

array([[3130,  289],
       [ 341, 2275]], dtype=int64)

In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91      3419
           1       0.89      0.87      0.88      2616

    accuracy                           0.90      6035
   macro avg       0.89      0.89      0.89      6035
weighted avg       0.90      0.90      0.90      6035



<h3>Using Tensorflow's Word2vec

<h3>Additional Tasks to perform on this project
<h5>->In the Neural Network try using different Optimizers along with changing learning rate to see its effect on accuracy
<h5>->Increase the number of epochs and apply a limit to it to get the best accuracy  
<h5>->Also use pre-made word2vec of Tensorflow